In [16]:
import argparse
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader

from skimage import io, transform
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
!pip install pandas


     |████████████████████████████████| 10.1 MB 5.8 MB/s eta 0:00:01


In [11]:
torch.cuda.is_available()
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32

# labels = {}
# normal_ids = listdir('./drive/My Drive/chest_xray/train/NORMAL')
# labels = dict(zip(normal_ids, np.zeros(len(normal_ids), dtype='int')))
# pneumonia_ids = listdir('./drive/My Drive/chest_xray/train/PNEUMONIA')
# labels.update(dict(zip(pneumonia_ids, np.ones(len(pneumonia_ids), dtype='int'))))
train_meta = pd.read_csv('../data/ISIC_2019_Training_Metadata.csv')
train_meta

,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male
...,...,...,...,...,...
25326,ISIC_0073247,85.0,head/neck,BCN_0003925,female
25327,ISIC_0073248,65.0,anterior torso,BCN_0001819,male
25328,ISIC_0073249,70.0,lower extremity,BCN_0001085,male
25329,ISIC_0073251,55.0,palms/soles,BCN_0002083,female


In [17]:
class ISICDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.names.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.df.iloc[idx, 3:]
        landmarks = np.array([landmarks])
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [18]:
root_dir = '../data/ISIC_2019_Training_Input/'
# train_dataset = torchvision.datasets.ImageFolder(root=data_path, 
#                                 transform=transforms.Compose([
#                                transforms.Resize(IMAGE_SIZE),
#                                #transforms.CenterCrop(IMAGE_SIZE),
#                                transforms.ToTensor(),
#                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                            ]))
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
#                                          shuffle=True)
isic_dataset = ISICDataset(csv_file='../data/ISIC_2019_Training_Metadata.csv', root_dir=root_dir)
train_iter = iter(isic_dataset)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img
